# Multiple Hypothesis Testing

In [0]:
import pandas as pd
import numpy as np

from scipy.stats import pearsonr
from statsmodels.sandbox.stats.multicomp import multipletests 

## Foodmart product sales 

In [0]:
sales = pd.read_csv('https://raw.githubusercontent.com/OzmundSedler/100-Days-Of-ML-Code/master/week_12/datasets/foodmart.sales.tsv', sep = '\t', header = 0, parse_dates = [2])

In [4]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83391 entries, 0 to 83390
Data columns (total 4 columns):
product_id    83391 non-null int64
store_id      83391 non-null int64
date          83391 non-null datetime64[ns]
sales         83391 non-null int64
dtypes: datetime64[ns](1), int64(3)
memory usage: 2.5 MB


In [3]:
sales.head()

,product_id,store_id,date,sales
0,4,6,1997-01-01,4
1,25,6,1997-01-01,3
2,48,6,1997-01-01,3
3,76,6,1997-01-01,4
4,119,6,1997-01-01,3


In [5]:
sales.describe()

,product_id,store_id,sales
count,83391.000000,83391.000000,83391.000000
mean,785.412994,13.166529,3.199062
std,447.310605,6.243769,1.091561
min,1.000000,2.000000,1.000000
25%,406.000000,7.000000,3.000000
50%,789.000000,13.000000,3.000000
75%,1171.000000,17.000000,4.000000
max,1559.000000,24.000000,15.000000


In [0]:
products = pd.read_csv('https://raw.githubusercontent.com/OzmundSedler/100-Days-Of-ML-Code/master/week_12/datasets/foodmart.products.tsv', sep = '\t', header = 0)

In [8]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 15 columns):
product_class_id      1560 non-null int64
product_id            1560 non-null int64
brand_name            1560 non-null object
product_name          1560 non-null object
SKU                   1560 non-null int64
SRP                   1560 non-null float64
gross_weight          1560 non-null float64
net_weight            1560 non-null float64
recyclable_package    1560 non-null bool
low_fat               1560 non-null bool
units_per_case        1560 non-null int64
cases_per_pallet      1560 non-null int64
shelf_width           1560 non-null float64
shelf_height          1560 non-null float64
shelf_depth           1560 non-null float64
dtypes: bool(2), float64(6), int64(5), object(2)
memory usage: 161.6+ KB


In [7]:
products.head()

,product_class_id,product_id,brand_name,product_name,SKU,SRP,gross_weight,net_weight,recyclable_package,low_fat,units_per_case,cases_per_pallet,shelf_width,shelf_height,shelf_depth
0,30,1,Washington,Washington Berry Juice,90748583674,2.85,8.39,6.39,False,False,30,14,16.9,12.60,7.40
1,52,2,Washington,Washington Mango Drink,96516502499,0.74,7.42,4.42,False,True,18,8,13.4,3.71,22.60
2,52,3,Washington,Washington Strawberry Drink,58427771925,0.83,13.10,11.10,True,True,17,13,14.4,11.00,7.77
3,19,4,Washington,Washington Cream Soda,64412155747,3.64,10.60,9.60,True,False,26,10,22.9,18.90,7.93
4,19,5,Washington,Washington Diet Soda,85561191439,2.19,6.66,4.65,True,False,7,10,20.7,21.90,19.20


In [9]:
products.describe()

,product_class_id,product_id,SKU,SRP,gross_weight,net_weight,units_per_case,cases_per_pallet,shelf_width,shelf_height,shelf_depth
count,1560.000000,1560.000000,1.560000e+03,1560.000000,1560.000000,1560.000000,1560.000000,1560.000000,1560.000000,1560.000000,1560.000000
mean,54.403205,780.500000,5.562930e+10,2.117282,13.804821,11.791308,18.908333,9.475641,12.900692,12.915103,12.933750
std,30.872364,450.477524,2.580380e+10,0.932779,4.631014,4.693747,10.254237,2.638963,5.763655,5.844180,5.603036
min,1.000000,1.000000,1.003103e+10,0.500000,6.000000,3.050000,1.000000,5.000000,3.010000,3.000000,3.000000
25%,30.000000,390.750000,3.345056e+10,1.417500,9.657500,7.705000,10.000000,7.000000,7.860000,7.867500,8.300000
50%,57.500000,780.500000,5.629308e+10,2.130000,13.600000,11.600000,19.000000,9.000000,12.900000,12.700000,12.850000
75%,79.000000,1170.250000,7.798317e+10,2.792500,17.725000,16.000000,28.000000,12.000000,17.625000,18.200000,17.600000
max,110.000000,1560.000000,9.998368e+10,3.980000,21.900000,20.800000,36.000000,14.000000,22.900000,22.900000,22.900000


In [0]:
sales = sales.merge(products[['product_id', 'product_name']], 
                    on = ['product_id'], how = 'inner')

In [11]:
sales.head()

,product_id,store_id,date,sales,product_name
0,4,6,1997-01-01,4,Washington Cream Soda
1,4,7,1997-01-05,3,Washington Cream Soda
2,4,6,1997-01-06,2,Washington Cream Soda
3,4,17,1997-01-11,2,Washington Cream Soda
4,4,24,1997-01-11,2,Washington Cream Soda


## Pearson correlation

In [0]:
sparse_sales = pd.pivot_table(sales, values='sales', index=['date', 'store_id'],
                     columns=['product_name'], fill_value = 0)

In [13]:
sparse_sales.head()

product_name         ADJ Rosy Sunglasses  ...  Washington Strawberry Drink
date       store_id                       ...                             
1997-01-01 6                           0  ...                            0
           14                          0  ...                            0
1997-01-02 11                          0  ...                            0
           23                          0  ...                            0
1997-01-03 7                           0  ...                            0

[5 rows x 1559 columns]

In [14]:
%%time 
corr_data = []

for i, lhs_column in enumerate(sparse_sales.columns):
    for j, rhs_column in enumerate(sparse_sales.columns):
        if i >= j:
            continue
        
        corr, p = pearsonr(sparse_sales[lhs_column], sparse_sales[rhs_column])
        corr_data.append([lhs_column, rhs_column, corr, p])

CPU times: user 5min 16s, sys: 483 ms, total: 5min 16s
Wall time: 5min 17s


In [0]:
sales_correlation = pd.DataFrame.from_records(corr_data)
sales_correlation.columns = ['product_A', 'product_B', 'corr', 'p']

In [16]:
sales_correlation.head()

,product_A,product_B,corr,p
0,ADJ Rosy Sunglasses,Akron City Map,0.076608,0.032414
1,ADJ Rosy Sunglasses,Akron Eyeglass Screwdriver,-0.006581,0.854396
2,ADJ Rosy Sunglasses,American Beef Bologna,0.038685,0.280546
3,ADJ Rosy Sunglasses,American Chicken Hot Dogs,0.041105,0.251529
4,ADJ Rosy Sunglasses,American Cole Slaw,-0.045887,0.200484


How many hypotheses about the lack of correlation are rejected without correction for multiple testing?

In [17]:
(sales_correlation.p < 0.05).value_counts()

False    982453
True     232008
Name: p, dtype: int64

## Correction for multiple hypothesis testing

### Holm’s Step-Down Procedure

In [0]:
reject, p_corrected, a1, a2 = multipletests(sales_correlation.p, 
                                            alpha = 0.05, 
                                            method = 'holm') 

In [0]:
sales_correlation['p_corrected'] = p_corrected
sales_correlation['reject'] = reject

In [20]:
sales_correlation.head()

,product_A,product_B,corr,p,p_corrected,reject
0,ADJ Rosy Sunglasses,Akron City Map,0.076608,0.032414,1.0,False
1,ADJ Rosy Sunglasses,Akron Eyeglass Screwdriver,-0.006581,0.854396,1.0,False
2,ADJ Rosy Sunglasses,American Beef Bologna,0.038685,0.280546,1.0,False
3,ADJ Rosy Sunglasses,American Chicken Hot Dogs,0.041105,0.251529,1.0,False
4,ADJ Rosy Sunglasses,American Cole Slaw,-0.045887,0.200484,1.0,False


In [21]:
sales_correlation.reject.value_counts()

False    1212733
True        1728
Name: reject, dtype: int64

In [22]:
sales_correlation[sales_correlation.reject == True].sort_values(by='corr', ascending=False)

,product_A,product_B,corr,p,p_corrected,reject
1063670,Just Right Vegetable Soup,Plato French Roast Coffee,0.340598,1.226033e-22,1.488970e-16,True
885574,Great Muffins,Nationeel Grape Fruit Roll,0.322176,2.688803e-20,3.265443e-14,True
473067,Club Low Fat Cottage Cheese,Skinner Strawberry Drink,0.306701,1.883995e-18,2.288034e-12,True
1181001,Robust Monthly Home Magazine,Tri-State Lemons,0.303269,4.674973e-18,5.677558e-12,True
1160248,Pleasant Regular Ramen Soup,Shady Lake Ravioli,0.298502,1.619119e-17,1.966350e-11,True
885261,Great Muffins,High Quality Copper Cleaner,0.297160,2.287551e-17,2.778130e-11,True
1151585,Plato Salt,Tri-State Onions,0.291224,1.032464e-16,1.253881e-10,True
1092548,Mighty Good Monthly Home Magazine,Tell Tale Baby Onion,0.290458,1.250831e-16,1.519077e-10,True
678154,Even Better Buttermilk,Sunset Toilet Paper,0.290046,1.386446e-16,1.683774e-10,True
914945,Hermanos Macintosh Apples,Plato French Roast Coffee,0.289095,1.757608e-16,2.134531e-10,True


### Benjamini-Hochberg Procedure

In [0]:
reject, p_corrected, a1, a2 = multipletests(sales_correlation.p, 
                                            alpha = 0.05, 
                                            method = 'fdr_bh') 

In [0]:
sales_correlation['p_corrected'] = p_corrected
sales_correlation['reject'] = reject

In [25]:
sales_correlation.head()

,product_A,product_B,corr,p,p_corrected,reject
0,ADJ Rosy Sunglasses,Akron City Map,0.076608,0.032414,0.203716,False
1,ADJ Rosy Sunglasses,Akron Eyeglass Screwdriver,-0.006581,0.854396,0.956078,False
2,ADJ Rosy Sunglasses,American Beef Bologna,0.038685,0.280546,0.630699,False
3,ADJ Rosy Sunglasses,American Chicken Hot Dogs,0.041105,0.251529,0.600790,False
4,ADJ Rosy Sunglasses,American Cole Slaw,-0.045887,0.200484,0.541916,False


In [26]:
sales_correlation.reject.value_counts()

False    1138407
True       76054
Name: reject, dtype: int64

In [27]:
sales_correlation[sales_correlation.reject == True].sort_values(by='corr', ascending=False)

,product_A,product_B,corr,p,p_corrected,reject
1063670,Just Right Vegetable Soup,Plato French Roast Coffee,0.340598,1.226033e-22,1.488970e-16,True
885574,Great Muffins,Nationeel Grape Fruit Roll,0.322176,2.688803e-20,1.632723e-14,True
473067,Club Low Fat Cottage Cheese,Skinner Strawberry Drink,0.306701,1.883995e-18,7.626793e-13,True
1181001,Robust Monthly Home Magazine,Tri-State Lemons,0.303269,4.674973e-18,1.419393e-12,True
1160248,Pleasant Regular Ramen Soup,Shady Lake Ravioli,0.298502,1.619119e-17,3.932713e-12,True
885261,Great Muffins,High Quality Copper Cleaner,0.297160,2.287551e-17,4.630236e-12,True
1151585,Plato Salt,Tri-State Onions,0.291224,1.032464e-16,1.791267e-11,True
1092548,Mighty Good Monthly Home Magazine,Tell Tale Baby Onion,0.290458,1.250831e-16,1.870872e-11,True
678154,Even Better Buttermilk,Sunset Toilet Paper,0.290046,1.386446e-16,1.870872e-11,True
914945,Hermanos Macintosh Apples,Plato French Roast Coffee,0.289095,1.757608e-16,2.134546e-11,True
